In [1]:
import requests
from bs4 import BeautifulSoup as bs
import pandas as pd
from tqdm import trange # to measure time

Before crawling, check **/robots.txt** from domain url.

(ex. www.inflearn.com/robots.txt)

In [2]:
base_url = "https://www.inflearn.com/pages/newyear-event-20200102"
response = requests.get(base_url)
response # response is 200 if normal

<Response [200]>

In [3]:
# response.text # loads every element in html page
soup = bs(response.text, 'html.parser')

In chrome dev tool, click 'copy selector' over the element which you want to crawl

main > section > div > div > div.chitchats > div.chitchat-list > div:nth-child(2) > div > div.body.edit-chitchat

In [4]:
content = soup.select("main > section > div > div > div.chitchats > div.chitchat-list > div")
content[-1] # last element means first comment

<div class="chitchat-item">
<figure class="image is-48x48">
<img alt="인프런" class="user_thumb is-rounded" src="https://cdn.inflearn.com/wp-content/uploads/avatars/17/b415d9fa24d186c4adf22ca9a49116b5-bpfull.png"/>
</figure>
<div class="content">
<div class="author">
<span class="author_name">인프런</span>
<time class="created_at" datetime="Sun Dec 29 2019 17:59:00 GMT+0900 (GMT+09:00)">⋅ 7개월 전</time>
<a class="update-chitchat edit-chitchat is-hidden" data-id="18594" type="button">저장</a>
<a class="hidden-editor edit-chitchat is-hidden" type="button">취소</a>
<a class="edit-chitchat no_cmt_reply" type="button">답글달기</a>
</div>
<div class="body edit-chitchat">인프런 0호 팀원이에요!
그동안 서비스 개발 때문에 js 를 많이 했었는데 앞으론 통계나 분석을 많이 하고 싶어서 파이썬을 공부하고 싶어요! 올해 파이썬 마스터가 되는걸로..
#관심강의: 남박사의 파이썬 활용</div>
<textarea class="textarea edit-chitchat is-hidden"></textarea>
<div class="summary_comments">
</div>
<div class="chitchats_and_editor is-hidden">
<div class="chitchat-comment-list">
</div>
<div class="cmt-editor">
<div c

In [5]:
chitchat = content[-1].select("div.body.edit-chitchat")[0].get_text(strip = True)
chitchat

'인프런 0호 팀원이에요!\n그동안 서비스 개발 때문에 js 를 많이 했었는데 앞으론 통계나 분석을 많이 하고 싶어서 파이썬을 공부하고 싶어요! 올해 파이썬 마스터가 되는걸로..\n#관심강의: 남박사의 파이썬 활용'

In [6]:
for i in range(5):
    chitchat = content[i].select("div.body.edit-chitchat")[0].get_text(strip = True)
    print("--------")
    print(chitchat)

--------
2020년 목표: 스프링 열심히 공부하서 직장에서 사랑받고 싶어요!!
관심 있는 강의 : 스프링 웹 MVC
--------
2020년 목표: C++ 열심히 공부해서, 학교에서 꼭 A 맞기..!! 
관심있는 강의: 따배씨 C++
--------
2020년 목표 : 리액트 공부하기
--------
40대 프로그래밍 시작! : 우리를 위한 프로그래밍 : 파이썬 중급
--------
2020년 목표 : 돌머리 비전공자가 멋진 전공자 되기!


In [7]:
content_cnt = len(content)
content_cnt

2447

In [8]:
events = []
# trange is from tqdm to measure time
for i in trange(content_cnt):
    chitchat = content[i].select("div.body.edit-chitchat")[0].get_text(strip = True)
    events.append(chitchat)

100%|████████████████████████████████████████████████████████| 2447/2447 [00:00<00:00, 5486.53it/s]


In [9]:
events[:5]

['2020년 목표: 스프링 열심히 공부하서 직장에서 사랑받고 싶어요!!\n관심 있는 강의 : 스프링 웹 MVC',
 '2020년 목표: C++ 열심히 공부해서, 학교에서 꼭 A 맞기..!! \n관심있는 강의: 따배씨 C++',
 '2020년 목표 : 리액트 공부하기',
 '40대 프로그래밍 시작! : 우리를 위한 프로그래밍 : 파이썬 중급',
 '2020년 목표 : 돌머리 비전공자가 멋진 전공자 되기!']

In [10]:
orgdf = pd.DataFrame({"text": events})
orgdf.shape

(2447, 1)

In [11]:
orgdf.to_csv("inflearn-event.csv", index=False)

In [15]:
df = pd.read_csv("inflearn-event.csv")

In [16]:
import numpy as np
import re
import matplotlib.pyplot as plt
#matplotlib settings
%matplotlib inline
%config InlineBackend.figure_format='retina'
plt.rc("font", family="Malgun Gothic")
plt.rc('axes', unicode_minus=False)

In [17]:
# delete duplicated comments. keeps last one
print(df.shape)
df = df.drop_duplicates(["text"], keep="last")
print(df.shape)

(2447, 1)
(2409, 1)


In [18]:
df["origin_text"] = df["text"]
df.head()

,text,origin_text
0,2020년 목표: 스프링 열심히 공부하서 직장에서 사랑받고 싶어요!!\n관심 있는 ...,2020년 목표: 스프링 열심히 공부하서 직장에서 사랑받고 싶어요!!\n관심 있는 ...
1,"2020년 목표: C++ 열심히 공부해서, 학교에서 꼭 A 맞기..!! \n관심있는...","2020년 목표: C++ 열심히 공부해서, 학교에서 꼭 A 맞기..!! \n관심있는..."
3,40대 프로그래밍 시작! : 우리를 위한 프로그래밍 : 파이썬 중급,40대 프로그래밍 시작! : 우리를 위한 프로그래밍 : 파이썬 중급
4,2020년 목표 : 돌머리 비전공자가 멋진 전공자 되기!,2020년 목표 : 돌머리 비전공자가 멋진 전공자 되기!
5,2020 년목표: 비전공자(경영)가 전공자(it) 되기!!!,2020 년목표: 비전공자(경영)가 전공자(it) 되기!!!


In [19]:
df["text"] = df["text"].str.lower()

In [21]:
df["text"] = df["text"].str.replace(
    "python", "파이썬").str.replace(
    "pandas", "판다스").str.replace(
    "javascript", "자바스크립트").str.replace(
    "java", "자바").str.replace(
    "react", "리액트")

In [27]:
df["course"] = df["text"].apply(lambda x: x.split("관심강의")[-1])
df["course"] = df["course"].apply(lambda x: x.split("관심 강의")[-1])
df["course"] = df["course"].apply(lambda x: x.split("관심 강좌")[-1])
df["course"] = df["course"].apply(lambda x: x.split("강의")[-1])
df["course"] = df["course"].str.replace(":", "")
df[["text", "course"]].tail()

,text,course
2442,"작년 한해도 일이 바쁘다, 야근해서 힘들다는 핑계로 김영한님의 jpa 강의를 또 스...",스프링 데이터 jpa
2443,저는 졸업을 1년 남기고 있는 컴퓨터공학과 학생입니다. 졸업 프로젝트로 웹/앱 개발...,스프링 부트 개념과 활용
2444,"에프터 이펙트를 써본 적은 있는데, 매번 기초만 배우다 말았어요. 이걸 할 줄 안다...",에펙 초보자도 만들 수 있는 유튜브 인트로 에프터이펙트 강좌
2445,저번에 인프런에서 페이스북 마케팅 강의를 듣고 많은 도움을 받았습니다. 마케팅 업무...,파이썬 입문 및 웹 크롤링을 활용한 다양한 자동화 어플리케이션 제작하기
2446,인프런 0호 팀원이에요!\n그동안 서비스 개발 때문에 js 를 많이 했었는데 앞으론...,남박사의 파이썬 활용


In [29]:
search_keyword = ['머신러닝', '딥러닝', '파이썬', '판다스', 
                  '크롤링', '시각화', '자바', '자바스크립트', 
                  '리액트', '스프링', 'c']
for keyword in search_keyword:
    df[keyword] = df["course"].str.contains(keyword)
df.head()

,text,origin_text,course,머신러닝,딥러닝,파이썬,판다스,크롤링,시각화,자바,자바스크립트,리액트,스프링,c
0,2020년 목표: 스프링 열심히 공부하서 직장에서 사랑받고 싶어요!!\n관심 있는 ...,2020년 목표: 스프링 열심히 공부하서 직장에서 사랑받고 싶어요!!\n관심 있는 ...,스프링 웹 mvc,False,False,False,False,False,False,False,False,False,True,True
1,"2020년 목표: c++ 열심히 공부해서, 학교에서 꼭 a 맞기..!! \n관심있는...","2020년 목표: C++ 열심히 공부해서, 학교에서 꼭 A 맞기..!! \n관심있는...",따배씨 c++,False,False,False,False,False,False,False,False,False,False,True
3,40대 프로그래밍 시작! : 우리를 위한 프로그래밍 : 파이썬 중급,40대 프로그래밍 시작! : 우리를 위한 프로그래밍 : 파이썬 중급,40대 프로그래밍 시작! 우리를 위한 프로그래밍 파이썬 중급,False,False,True,False,False,False,False,False,False,False,False
4,2020년 목표 : 돌머리 비전공자가 멋진 전공자 되기!,2020년 목표 : 돌머리 비전공자가 멋진 전공자 되기!,2020년 목표 돌머리 비전공자가 멋진 전공자 되기!,False,False,False,False,False,False,False,False,False,False,False
5,2020 년목표: 비전공자(경영)가 전공자(it) 되기!!!,2020 년목표: 비전공자(경영)가 전공자(it) 되기!!!,2020 년목표 비전공자(경영)가 전공자(it) 되기!!!,False,False,False,False,False,False,False,False,False,False,False


In [31]:
df_python = df[df["text"].str.contains("파이썬|판다스")].copy()
df_python.shape

(430, 14)

In [32]:
df[search_keyword]

,머신러닝,딥러닝,파이썬,판다스,크롤링,시각화,자바,자바스크립트,리액트,스프링,c
0,False,False,False,False,False,False,False,False,False,True,True
1,False,False,False,False,False,False,False,False,False,False,True
3,False,False,True,False,False,False,False,False,False,False,False
4,False,False,False,False,False,False,False,False,False,False,False
5,False,False,False,False,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...
2442,False,False,False,False,False,False,False,False,False,True,False
2443,False,False,False,False,False,False,False,False,False,True,False
2444,False,False,False,False,False,False,False,False,False,False,False
2445,False,False,True,False,True,False,False,False,False,False,False


In [34]:
df[search_keyword].sum().sort_values(ascending=False)

c         380
파이썬       375
스프링       346
자바        318
자바스크립트    130
리액트       124
머신러닝      124
크롤링        55
딥러닝        46
시각화        26
판다스         5
dtype: int64

In [35]:
text = df.loc[(df["딥러닝"] == True), "text"]
for t in text:
    print("-"*20)
    print(t)

--------------------
딥러닝을 공부하여 인공지능에 한걸음 다가가고 싶습니다~~!!
#관심강의 : [opencv] 파이썬 딥러닝 영상처리 프로젝트 - 손흥민을 찾아라!
--------------------
금융인을 꿈꾸는 인문대생인데 인프런을 통해 그 꿈에 날개를 달 수 있으면 좋겠습니다!!
#선대 #머신러닝 #딥러닝 #nlp #finance
--------------------
올해 새롭게 딥러닝에 대한 공부를 시작하고자 합니다!
#관심분야 : 밑바닥부터 시작하는 머신러닝 입문,  공공 데이터 분석
--------------------
딥러닝에 관심이 생겨 구글링 중 인프런에 좋은 강의를 발견했습니다. 
열심히 ai 공부하고 싶습니다 
#관심강의: 모두를 위한 딥러닝 - 기본적인 머신러닝과 딥러닝 강좌
--------------------
평생 개발자 하렵니다. 인프런에서 잘 배워서!
#관심강의: 파이썬 인공지능, 머신러닝, 딥러닝 제대로 배워요!, 데이터 과학부터 인공지능까지!
--------------------
바빠서 밀어 두었던 인공지능과 데이터 사이언스를 다시 제대로 공부를 하려 합니다.
# [opencv] 파이썬 딥러닝 영상처리 프로젝트 - 손흥민을 찾아라!
--------------------
크라우드펀딩한 강의 듣고 자연어처리 할 수 있는 능력이 올해 안에 생기길...ㅜ 해야할 것들이 너무 많아서 버겁지만 그래도 도전!

#관심강의 : 자연어처리, 머신러닝, 딥러닝, 대학수학, 파이썬, 파이토치, 케라스
--------------------
파이썬 로드맵에 따라 열심히 공부해서 데이터 사이언스 skill up 하려고 합니다.
책으로 공부하면 진도도 잘 안 나갔는데 강의를 들으니 하루하루 일정하게 진도도 나가고 이해도 쉽네요.
#[opencv] 파이썬 딥러닝 영상처리 프로젝트 - 손흥민을 찾아라!
--------------------
올해는 딥러닝을 활용한 토이 프로젝트에 도전해보려합니다!
--------------------
20